### Citibike Analysis

In [1]:
import os
from dotenv import load_dotenv

from google.cloud import bigquery
from google.api_core.exceptions import NotFound, BadRequest


import numpy as np


In [2]:
load_dotenv()

True

In [3]:
client = bigquery.Client()

bq_project_id = os.getenv('GCP_PROJECT_ID')
bq_dataset_name = os.getenv('BQ_DATASET')
bq_table_name = os.getenv('BQ_TABLE')

table_red = f'{bq_project_id}.{bq_dataset_name}.{bq_table_name}'
cleaned_table_ref = f'{bq_project_id}.{bq_dataset_name}.{bq_table_name}_cleaned'

In [4]:
query = f"""
    SELECT *
    FROM `{bq_dataset_name}.{bq_table_name}`
    LIMIT 10
"""

df = client.query(query).to_dataframe()
df.head()

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,cleaned_start_station_id,cleaned_start_station_name
0,3F97AB2320337E97,classic_bike,2023-12-31 19:17:21.067000+00:00,2024-01-01 20:16:59.127000+00:00,W 37 St & Broadway,6441.06,None,nan,40.752271,-73.987706,NaN,NaN,casual,None,None
1,36D0948945B60A14,classic_bike,2023-12-31 06:25:55.501000+00:00,2024-01-01 07:25:47.279000+00:00,W 37 St & Broadway,6441.06,None,nan,40.752271,-73.987706,NaN,NaN,casual,None,None
2,B82F865E66B13DA3,classic_bike,2023-12-31 14:50:10.465000+00:00,2024-01-01 15:50:04.927000+00:00,Central Park West & W 76 St,7253.04,None,nan,40.778968,-73.973747,NaN,NaN,casual,None,None
3,068B974DABC0B2B8,classic_bike,2023-12-31 12:53:15.228000+00:00,2024-01-01 13:52:56.073000+00:00,Soissons Landing,4590.01,None,nan,40.692317,-74.014866,NaN,NaN,casual,None,None
4,9FF2EF018137DE33,classic_bike,2023-12-31 12:11:27.022000+00:00,2024-01-01 13:11:20.711000+00:00,Bergen St & 4 Ave,4322.06,None,nan,40.682564,-73.979898,NaN,NaN,casual,None,None


In [5]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'cleaned_start_station_id',
       'cleaned_start_station_name'],
      dtype='object')

In [6]:
df.dtypes

ride_id                                    object
rideable_type                              object
started_at                    datetime64[us, UTC]
ended_at                      datetime64[us, UTC]
start_station_name                         object
start_station_id                           object
end_station_name                           object
end_station_id                             object
start_lat                                 float64
start_lng                                 float64
end_lat                                   float64
end_lng                                   float64
member_casual                              object
cleaned_start_station_id                   object
cleaned_start_station_name                 object
dtype: object

In [7]:
query = f"""
    SELECT 
        COUNT(*) - COUNT(ride_id) as ride_id_null,
        COUNT(*) - COUNT(rideable_type) as rideable_type_null,
        COUNT(*) - COUNT(started_at) as started_at_null,
        COUNT(*) - COUNT(ended_at) as ended_at_null,
        COUNT(*) - COUNT(start_station_name) as start_station_name_null,
        COUNT(*) - COUNT(start_station_id) as start_station_idnull,
        COUNT(*) - COUNT(end_station_name) as end_station_name_null,
        COUNT(*) - COUNT(end_station_id) as end_station_id_null,
        COUNT(*) - COUNT(start_lat) as start_lat_null,
        COUNT(*) - COUNT(start_lng) as start_lng_null,
        COUNT(*) - COUNT(end_lat) as end_lat_null,
        COUNT(*) - COUNT(end_lng) as end_lng_null,
        COUNT(*) - COUNT(member_casual) as member_casual_null
    FROM `{bq_dataset_name}.{bq_table_name}`
"""

df = client.query(query).to_dataframe()
df.T

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,0
ride_id_null,0
rideable_type_null,0
started_at_null,0
ended_at_null,0
start_station_name_null,35253
start_station_idnull,0
end_station_name_null,152745
end_station_id_null,0
start_lat_null,0
start_lng_null,0


In [93]:
query = f"""
    SELECT *
    FROM `{bq_dataset_name}.{bq_table_name}`
    WHERE start_station_name IS NULL
    LIMIT 10
"""

df = client.query(query).to_dataframe()
df

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,cleaned_start_station_id,cleaned_start_station_name
0,74AFA20DD715A959,electric_bike,2024-01-05 11:31:21.225000+00:00,2024-01-05 12:33:04.715000+00:00,None,nan,Park Circle & East Dr,3344.02,40.65,-73.97,40.651566,-73.972212,casual,None,None
1,C485FFB302106109,electric_bike,2024-01-30 22:33:30.300000+00:00,2024-01-30 22:40:43.339000+00:00,None,nan,Caton Ave & Bedford Ave,3368.04,40.67,-73.95,40.652370,-73.956230,casual,None,None
2,D227111A050E09A8,electric_bike,2024-01-21 18:13:59.990000+00:00,2024-01-21 18:21:10.019000+00:00,None,nan,Prospect Park SW & Vanderbilt St,3457.03,40.67,-73.98,40.654820,-73.973217,casual,None,None
3,1DBB7468137BA86F,electric_bike,2024-01-23 17:38:07.439000+00:00,2024-01-23 17:49:45.748000+00:00,None,nan,Washington Ave & Empire Blvd,3704.01,40.68,-73.95,40.663140,-73.960570,casual,None,None
4,04F820511BC4B542,electric_bike,2024-01-05 12:47:02.253000+00:00,2024-01-05 12:51:59.618000+00:00,None,nan,6 St & 7 Ave,3834.10,40.66,-73.99,40.668663,-73.979881,casual,None,None
5,C60EFE52BE6EF30B,electric_bike,2024-01-31 19:52:12.226000+00:00,2024-01-31 20:05:40.503000+00:00,None,nan,Lewis Ave & Decatur St,4237.01,40.68,-73.98,40.681460,-73.934903,casual,None,None
6,A03CFDE86F5B1BD1,electric_bike,2024-01-04 07:12:18.723000+00:00,2024-01-04 07:16:48.271000+00:00,None,nan,Halsey St & Tompkins Ave,4319.07,40.69,-73.95,40.682369,-73.944118,casual,None,None
7,91E4D1BAA3A81F62,electric_bike,2024-01-30 01:25:36.255000+00:00,2024-01-30 01:27:59.127000+00:00,None,nan,Halsey St & Tompkins Ave,4319.07,40.68,-73.94,40.682369,-73.944118,casual,None,None
8,9EA6AA43D27E8CAA,electric_bike,2024-01-26 12:42:16.789000+00:00,2024-01-26 12:49:46.538000+00:00,None,nan,Lexington Ave & Classon Ave,4452.03,40.69,-73.96,40.686768,-73.959282,casual,None,None
9,AB8561213F626329,electric_bike,2024-01-28 08:54:08.772000+00:00,2024-01-28 08:59:40.743000+00:00,None,nan,Court St & State St,4488.08,40.70,-73.99,40.690238,-73.992031,casual,None,None


In [107]:
query = f"""
    SELECT *
    FROM `{bq_dataset_name}.{bq_table_name}`
    WHERE end_station_name IS NULL
    LIMIT 10
"""

df = client.query(query).to_dataframe()
df

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,cleaned_start_station_id,cleaned_start_station_name
0,17C766D3A655BF8A,classic_bike,2024-01-04 22:51:30.140000+00:00,2024-01-05 23:51:26.067000+00:00,Caton Ave & Bedford Ave,3368.04,None,nan,40.652370,-73.956230,NaN,NaN,member,None,None
1,13EF25F33B63EA85,classic_bike,2024-01-05 08:13:53.376000+00:00,2024-01-06 09:13:40.930000+00:00,14 St & 7 Ave,3731.11,None,nan,40.663779,-73.983968,NaN,NaN,member,None,None
2,998C13F35E3AA327,classic_bike,2024-01-15 14:48:00.507000+00:00,2024-01-16 15:47:53.855000+00:00,Halsey St & Tompkins Ave,4319.07,None,nan,40.682369,-73.944118,NaN,NaN,member,None,None
3,73A4227F8817C643,classic_bike,2024-01-02 07:53:15.249000+00:00,2024-01-03 08:53:09.992000+00:00,Carroll St & Columbia St,4348.07,None,nan,40.683046,-74.003486,NaN,NaN,member,None,None
4,A482F744AD22ECE9,classic_bike,2024-01-15 15:49:10.968000+00:00,2024-01-16 16:48:51.174000+00:00,Atlantic Ave & Fort Greene Pl,4354.07,None,nan,40.683826,-73.976323,NaN,NaN,member,None,None
5,5B497FF3315AD009,classic_bike,2024-01-23 09:41:56.595000+00:00,2024-01-24 10:41:51.461000+00:00,Warren St & Smith St,4371.01,None,nan,40.685424,-73.991278,NaN,NaN,member,None,None
6,38689AEFFD0D8270,classic_bike,2024-01-06 15:40:21.941000+00:00,2024-01-07 16:40:17.865000+00:00,Putnam Ave & Throop Ave,4392.04,None,nan,40.685153,-73.941110,NaN,NaN,member,None,None
7,6CBC7FF21FB45CA3,classic_bike,2024-01-27 12:04:07.207000+00:00,2024-01-28 13:04:02.710000+00:00,Warren St & Court St,4413.08,None,nan,40.686371,-73.993833,NaN,NaN,member,None,None
8,B36E561B23B3D403,classic_bike,2024-01-29 15:33:28.034000+00:00,2024-01-30 16:33:19.773000+00:00,Monroe St & Tompkins Ave,4434.06,None,nan,40.686203,-73.944694,40.68,-73.95,member,None,None
9,32BD4CFEEC4F53EC,classic_bike,2024-01-13 10:51:14.782000+00:00,2024-01-14 11:51:06.763000+00:00,3 Ave & Schermerhorn St,4437.01,None,nan,40.686832,-73.979677,NaN,NaN,member,None,None


In [101]:
query = f"""
    SELECT COUNT(*) as cnt
    FROM `{bq_dataset_name}.{bq_table_name}`
    WHERE start_station_id = 'nan' OR end_station_id = 'nan'
"""

nan_cnt = client.query(query).to_dataframe()['cnt'][0]
nan_cnt

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


np.int64(182206)

In [103]:
query = f"""
    CREATE OR REPLACE TABLE {cleaned_table_ref}
    PARTITION BY DATE_TRUNC(started_at, MONTH)
    CLUSTER BY member_casual, rideable_type 
        AS
            SELECT * FROM `{bq_dataset_name}.{bq_table_name}`
            WHERE start_station_id != 'nan' AND end_station_id != 'nan'
"""

job = client.query(query)
job.result()


In [109]:
query = f"""
    SELECT start_station_id, COUNT(*)
    FROM {cleaned_table_ref}
    WHERE start_station_id IS NULL OR SAFE_CAST(start_station_id AS FLOAT64) IS NULL
    GROUP BY start_station_id;
"""

df = client.query(query).to_dataframe()
df

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_id,f0_
0,JC098,1
1,JC066,3
2,Lab - NYC,6
3,HB305,4
4,HB202,3
...,...,...
72,HB303,1
73,HB103,3
74,JC022,2
75,JC105,4


In [ ]:
# Regex patterns to exclude
regex_patterns = [
    r'^[A-Za-z]{2}\d{3}$',  # Two letters + 3 digits
    r'^SYS\d{3}$',          # SYS + 3 digits
    r'(?i)Demo',            # Contains "Demo"
    r'(?i)Lab - NYC',       # Contains "Lab - NYC"
    r'(?i)Morgan'           # Contains "Morgan"
]

# Combine patterns using alternation (OR)
combined_pattern = '|'.join(regex_patterns)

# Create the query excluding the combined regex patterns
query = f"""
CREATE OR REPLACE TABLE `{bq_dataset_name}.{bq_table_name}_cleaned`
PARTITION BY DATE_TRUNC(started_at, MONTH)
CLUSTER BY member_casual, rideable_type

 AS

  SELECT * EXCEPT(start_station_id, start_station_name),
  -- Clean start_station_id
  CASE
    WHEN REGEXP_CONTAINS(start_station_id, r'^\\d+\\.\\d+_$') THEN REPLACE(start_station_id, '_', '')
    WHEN CONTAINS_SUBSTR(start_station_id, 'Pillar') THEN REGEXP_REPLACE(start_station_id, r'_Pillar$', '')
    ELSE start_station_id
  END AS start_station_id,

  -- Clean start_station_name only if ID contains 'Pillar'
  CASE
    WHEN CONTAINS_SUBSTR(start_station_id, 'Pillar') THEN TRIM(REGEXP_REPLACE(start_station_name, r'(?i)\s*Pillar\s*', ' '))
    ELSE TRIM(REGEXP_REPLACE(start_station_name, r'[\s\u00A0\u200B]+', ' ')) 
  END AS start_station_name

FROM `{bq_dataset_name}.{bq_table_name}`
WHERE NOT REGEXP_CONTAINS(start_station_id, r"{combined_pattern}")
"""

job = client.query(query)
job.result()


<>:37: SyntaxWarning: invalid escape sequence '\s'
<>:37: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_15462/3094358261.py:37: SyntaxWarning: invalid escape sequence '\s'
  """


In [61]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM `{cleaned_table_ref}`
    WHERE start_station_id IS NULL OR SAFE_CAST(start_station_id AS FLOAT64) IS NULL
    GROUP BY start_station_name, start_station_id;
"""

df = client.query(query).to_dataframe()
df

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_


In [62]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM {cleaned_table_ref}
    GROUP BY start_station_name, start_station_id
"""

df = client.query(query).to_dataframe()
df[df['start_station_id'].duplicated(keep=False)].sort_values(by='start_station_id')

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_
26,Eastern Pkwy & St Marks Ave,3982.01,1169
570,Eastern Pkwy\t& St Marks Ave,3982.01,1557
1808,Bridge St & Front St,4968.03,19105
1744,Bridge St & Water St,4968.03,26853
573,Morton St & Washington St,5772.05,30999
773,Morton St & Greenwich St,5772.05,39300
2145,34th Ave & Vernon Blvd,6873.01,12397
435,34 Ave & Vernon Blvd,6873.01,3864
453,Central Park West & W 68 St,7079.06,82351
2187,Central Park W & W 68 St,7079.06,26746


In [63]:
query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_id = CAST(FORMAT('%.2f', CAST(start_station_id AS FLOAT64)) AS STRING)
    WHERE TRUE
"""
job = client.query(query)
job.result()

In [64]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM {cleaned_table_ref}
    GROUP BY start_station_name, start_station_id
"""

df = client.query(query).to_dataframe()
df[df['start_station_id'].duplicated(keep=False)].sort_values(by='start_station_id')

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_
66,Eastern Pkwy & St Marks Ave,3982.01,1169
1155,Eastern Pkwy\t& St Marks Ave,3982.01,1557
1547,Bridge St & Water St,4968.03,26853
943,Bridge St & Front St,4968.03,19105
1119,Morton St & Greenwich St,5772.05,39300
363,Morton St & Washington St,5772.05,30999
1946,34 Ave & Vernon Blvd,6873.01,3864
1566,34th Ave & Vernon Blvd,6873.01,12397
1014,Central Park W & W 68 St,7079.06,26746
254,Central Park West & W 68 St,7079.06,82351


In [65]:
convert_names = {
    'West': 'W',
    'Fort': 'Ft',
    'Av': 'Ave',
    'Ichan Stadium': 'Icahn Stadium'
}

sql_expr = "start_station_name"
for old, new in convert_names.items():
    sql_expr = f"REGEXP_REPLACE({sql_expr}, r'\\b{old}\\b', '{new}')"

query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_name = {sql_expr}
    WHERE TRUE
"""

job = client.query(query)
job.result()

In [66]:
pattern = r'\\t'

query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_name = REGEXP_REPLACE(start_station_name, r'{pattern}', ' ')
    WHERE REGEXP_CONTAINS(start_station_name, r'{pattern}')
"""

job = client.query(query)
job.result()

In [67]:
pattern = r'(\d+)(st|nd|rd|th)\b'
replace = r'\1'

query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_id = REGEXP_REPLACE(start_station_name, r'{pattern}', r'{replace}')
    WHERE REGEXP_CONTAINS(start_station_name, r'{pattern}')
"""

job = client.query(query)
job.result()

In [68]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM {cleaned_table_ref}
    GROUP BY start_station_name, start_station_id
"""

df = client.query(query).to_dataframe()
df[df['start_station_id'].duplicated(keep=False)].sort_values(by='start_station_id')

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_
846,Bridge St & Front St,4968.03,19105
1434,Bridge St & Water St,4968.03,26853
443,Morton St & Greenwich St,5772.05,39300
1401,Morton St & Washington St,5772.05,30999
1740,3 Ave & E 82 St,7154.10,41574
2217,3 Ave & E 81 St,7154.10,21819


In [76]:
convert_names = {
    'Bridge St & Water St': 'Bridge St & Front St',
    'Morton St & Washington St': 'Morton St & Greenwich St',
    '3 Ave & E 81 St': '3 Ave & E 82 St'
}

case_statements = []
for old, new in convert_names.items():
    case_statements.append(f"WHEN start_station_name = '{old}' THEN '{new}'")

query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_name = CASE {' '.join(case_statements)} ELSE start_station_name END
    WHERE start_station_name IN ({', '.join(f"'{old}'" for old in convert_names.keys())})
"""

job = client.query(query)
job.result()

In [77]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM {cleaned_table_ref}
    GROUP BY start_station_name, start_station_id
"""

df = client.query(query).to_dataframe()
df[df['start_station_id'].duplicated(keep=False)].sort_values(by='start_station_id')

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_
